In [1]:
import numpy as np
import os
from multiprocessing import Pool, cpu_count
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import read_data as rd

2024-06-02 16:31:52.115871: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 16:31:52.777421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load data
dataset_dir = '/home/ecappiell/datasets/full'
data_arrays, labels, class_ids = rd.process_mafaulda_data(dataset_dir)

In [3]:
def downsample_data(data_arrays, original_sampling_rate, target_sampling_rate):
    # Calculate the downsampling factor
    downsampling_factor = int(original_sampling_rate / target_sampling_rate)

    # Reshape data to have each row represent one channel
    num_channels, num_samples, num_features = data_arrays.shape
    data_arrays_reshaped = data_arrays.reshape(num_channels, num_samples * num_features)

    # Downsample each channel
    downsampled_data = []
    for channel_data in data_arrays_reshaped:
        downsampled_channel = []
        for i in range(0, len(channel_data), downsampling_factor):
            # Take the first sample in each downsampling interval
            downsampled_channel.append(channel_data[i])
        downsampled_data.append(downsampled_channel)

    return np.array(downsampled_data).reshape(num_channels, -1, num_features)

In [5]:
# Original sampling rate (in Hz)
original_sampling_rate = 50 * 10**3  # 50 kHz

# Target sampling rate (in Hz)
target_sampling_rate = 1 * 10**3  # 1 kHz

# Downsample the data
data_downsampled = downsample_data(data_arrays, original_sampling_rate, target_sampling_rate)